In [1]:
using Distributed

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [ ]:
rmprocs(nworkers())

In [14]:
list = [i for i in 1:10 for _ in 1:rand(1:10)]

48-element Array{Int64,1}:
  1
  1
  2
  2
  2
  3
  3
  3
  3
  3
  3
  4
  4
  ⋮
  8
  9
  9
  9
  9
  9
  9
 10
 10
 10
 10
 10

In [25]:
c = RemoteChannel(()->Channel(0))
d = RemoteChannel(()->Channel(0))

@everywhere function do_work(c, d)
    while true
        lc = try take!(c) catch nothing end
        if lc === nothing
            close(c)
            break
        end
        
        num = 0
        count = 0
        while true
            val = try take!(lc) catch nothing end
            if val === nothing
                close(lc)
                break
            end
            
            num = val
            count += 1
            put!(d, val)
        end
        #put!(d, (num, count))
    end
    put!(d, nothing)
end

function make_jobs(c, list)
    ts = []
    start = 1
    
    while start <= length(list)
        v = list[start]
        next = findnext(!isequal(v), list, start)
        if next === nothing
            next = length(list) + 1
        end
        lc = RemoteChannel(()->Channel(0))
        put!(c, lc)
        t = Task(() -> begin
                for i in list[start:next-1]
                    put!(lc, i)
                    yield()
                end
                put!(lc, nothing)
            end)
        push!(ts, t)
        schedule(t)
        yield()
        start = next
    end
    println("ok")
    for t in ts
        wait(t)
        #println("Task $t done")
    end
    put!(c, nothing)
end

for p in workers()
    remote_do(do_work, p, c, d)
end

@async make_jobs(c, list)
        
while true
    t = try take!(d) catch nothing end
    if t === nothing
        #close(d)
        break
    end
    println(t)
end

3
1
3
1
3
3
2
5
3
2
5
3
2
5
5
6
7
5
6
7
4
6
7
4
6
8
7
4
6
8
4
6
9
4
6
9
10
6
9
10
6
9
10
6
9
ok
10
9
10
